In [89]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [90]:
X = pd.read_csv("TrainningInput_Dataset.csv")
X.drop(columns=X.columns[0],axis=1,inplace=True) # just bc in my dataset i have a unused columns at the beginning
print(X.shape)

y = pd.read_csv("TrainningOutput_Dataset.csv")
y.drop(columns=y.columns[0],axis=1,inplace=True)
print(y.shape)

(100, 12)
(100, 2)


In [91]:
# Shuffle the data
X = X.sample(n=X.shape[0])
y = y.loc[X.index]
print(y.head())
print(X.head())


     Id  Category
74   75         0
55   56         1
99  100         4
90   91         4
0     1         2
     Id  ED_RV_volume  ED_LV_volume  ED_MY_volume  ES_RV_volume  ES_LV_volume  \
74   75          7609          7053          4910          4442          3421   
55   56          7583          9655          6112          4247          6071   
99  100         10715          4141          3968          6506          1744   
90   91          7836          5168          2312          4676          2346   
0     1          5723         12104          6728          2439          9241   

    ES_MY_volume  RV_DIFF  LV_DIFF  MY_DIFF  Height  Weight  
74          6367     3167     3632     1457   179.0    93.0  
55          7254     3336     3584     1142   170.0    62.0  
99          4733     4209     2397      765   165.0    63.0  
90          2424     3160     2822      112   162.0    53.0  
0           7990     3284     2863     1262   184.0    95.0  


In [67]:
# Here create other feature from the original ones. Maybe compute imc of each person and divide each feature by that. Then remove the feature height and weight.

X["Height"] = X["Height"]/100
X["IMC"] = X["Weight"].div(X["Height"]**2,axis=0)
Xdiv = X.drop(columns=["Id"]).div(X["IMC"],axis=0)
Xdiv["Id"] = X["Id"]
print(Xdiv)

X = Xdiv.drop(columns=["Height","Weight","IMC"])

    ED_RV_volume  ED_LV_volume  ED_MY_volume  ES_RV_volume  ES_LV_volume  \
21    196.279286    235.172143    230.958750    109.613036     82.452857   
69    125.546800    115.877200     84.782800     47.716000     43.544800   
72    201.178828    163.816536    125.741224    107.238333     60.642422   
24    317.676468    329.528415    348.986283     99.246205    121.990127   
56   1013.457812    911.055469    978.966406    528.664062    688.871094   
..           ...           ...           ...           ...           ...   
41    159.394327    195.230255    118.597818     66.084073    130.657164   
10    329.554286    497.895429    278.732571    276.140571    436.613143   
46    175.157561    286.621463    187.485366     85.188293    238.653659   
94    621.518092    362.665658    180.795493    397.699934    161.881283   
77    383.157024    369.548512    299.020565    169.250774    136.777768   

    ES_MY_volume     RV_DIFF     LV_DIFF     MY_DIFF    Height  Weight  IMC  \
21    26

In [92]:
pipeline = Pipeline([
    ("normaliser" , MinMaxScaler()),
    ("classifier", RandomForestClassifier()),
])
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'normaliser', 'classifier', 'normaliser__clip', 'normaliser__copy', 'normaliser__feature_range', 'classifier__bootstrap', 'classifier__ccp_alpha', 'classifier__class_weight', 'classifier__criterion', 'classifier__max_depth', 'classifier__max_features', 'classifier__max_leaf_nodes', 'classifier__max_samples', 'classifier__min_impurity_decrease', 'classifier__min_samples_leaf', 'classifier__min_samples_split', 'classifier__min_weight_fraction_leaf', 'classifier__monotonic_cst', 'classifier__n_estimators', 'classifier__n_jobs', 'classifier__oob_score', 'classifier__random_state', 'classifier__verbose', 'classifier__warm_start'])

In [93]:
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_features': ['sqrt', 'log2'],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
}

In [94]:
grid_search = GridSearchCV(pipeline,param_grid=param_grid,cv=5,verbose=3,return_train_score=True)
grid_search.fit(X.drop(columns = ["Id"]),y["Category"])

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV 1/5] END classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=100;, score=(train=1.000, test=0.950) total time=   1.2s
[CV 2/5] END classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=100;, score=(train=1.000, test=0.700) total time=   0.9s
[CV 3/5] END classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=100;, score=(train=1.000, test=0.800) total time=   0.5s
[CV 4/5] END classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=100;, score=(train=1.000, test=0.900) total time=   0.6s
[CV 5/5] END classifier__max_depth=None, classifier__max_fea

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('normaliser', MinMaxScaler()),
                                       ('classifier',
                                        RandomForestClassifier())]),
             param_grid={'classifier__max_depth': [None, 10, 20, 30],
                         'classifier__max_features': ['sqrt', 'log2'],
                         'classifier__min_samples_leaf': [1, 2, 4],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [100, 200, 300]},
             return_train_score=True, verbose=3)

The model is fitted and now we want to properly evaluate the results. 
We select only the result from the best_params founded by the search. (Reminder that the best params are the one that provided the best mean validation score)

In [95]:
best_params = grid_search.best_params_
print(best_params)
results = grid_search.cv_results_

# Only keep the cv result for the best paramaters.
best_idx = results['params'].index(best_params)

mean_train_score = results['mean_train_score'][best_idx]
mean_valid_score = results['mean_test_score'][best_idx]
std_train_score = results['std_train_score'][best_idx]
std_valid_score = results['std_test_score'][best_idx]
print(mean_valid_score,std_valid_score)
print(mean_train_score,std_train_score)

{'classifier__max_depth': 20, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}
0.9 0.05477225575051658
1.0 0.0


In [ ]:
print(results)

In [96]:
f_importance = grid_search.best_estimator_.named_steps["classifier"].feature_importances_
f_name = grid_search.best_estimator_.named_steps["normaliser"].get_feature_names_out()
feature_importance = pd.DataFrame({
    "feature": f_name,
    "importance": f_importance
})
feature_importance.sort_values("importance", ascending=False, inplace=True)
print(feature_importance)

         feature  importance
4   ES_LV_volume    0.244536
3   ES_RV_volume    0.153332
1   ED_LV_volume    0.112343
5   ES_MY_volume    0.089794
0   ED_RV_volume    0.086098
7        LV_DIFF    0.075899
2   ED_MY_volume    0.074280
6        RV_DIFF    0.058218
8        MY_DIFF    0.037750
9         Height    0.034318
10        Weight    0.033431


Below two cells to save the results : the model and the description of the method used.

In [99]:
description = "MinmaxScaler + Randomforest with Gridsearch" 
other_params = "The features are just the volume of each segmentation, also the difference between the 2 timesteps"
feature_used = f_name
informationDict = {
    "description": description,
    "model parameters" : best_params,
    "features used" : feature_used,
    "other parms" : other_params,
    
}

In [100]:
import joblib
from datetime import datetime
import os

# Create a timestamp
currentDateTime = datetime.now()

# Get the base directory (current directory)
BASE_DIR = os.getcwd()
print(f"BASE_DIR: {BASE_DIR}")
RESULT_DIR = os.path.join(BASE_DIR,"output")
if not os.path.exists(RESULT_DIR):
    os.mkdir(RESULT_DIR)
    print(f"Directory created: {RESULT_DIR}")

# Create a folder named 'pipeline_<timestamp>' in the current directory
dir_name = 'pipeline_' + currentDateTime.strftime("%H-%M-%S")
dir_path = os.path.join(RESULT_DIR, dir_name)

# If the directory doesn't exist, create it
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
    print(f"Directory created: {dir_path}")

# Save the model inside this new folder
model_filename = 'pipeline_' + currentDateTime.strftime("%H-%M-%S") + '.pkl'
model_path = os.path.join(dir_path, model_filename)

# This is where you'd have your model defined
joblib.dump(grid_search, model_path)
print(f"Model saved to: {model_path}")


# Saving feature importance : 
feature_importance_filename ='pipeline_' + currentDateTime.strftime("%H-%M-%S") + '.csv' 
feature_importance_dir = os.path.join(dir_path,feature_importance_filename)
feature_importance.to_csv(feature_importance_dir)


# SAVING Description 
dict_filename = 'params.txt'
dict_path = os.path.join(dir_path, dict_filename)

with open(dict_path, 'w') as f:
    for key, val in informationDict.items():
        f.write(f"{key} : {val}\n")
print(f"Information about the model saved to: {dict_path}")


BASE_DIR: /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction
Directory created: /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/output/pipeline_18-19-26
Model saved to: /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/output/pipeline_18-19-26/pipeline_18-19-26.pkl
Information about the model saved to: /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/output/pipeline_18-19-26/params.txt
